In [40]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [28]:
!pip install keras-core --upgrade
!pip install -q keras-nlp --upgrade

# This sample uses Keras Core, the multi-backend version of Keras.
# The selected backend is TensorFlow (other supported backends are 'jax' and 'torch')
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'

In [41]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import keras_core as keras
import keras_nlp
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis,QuadraticDiscriminantAnalysis
print("TensorFlow version:", tf.__version__)
print("KerasNLP version:", keras_nlp.__version__)

TensorFlow version: 2.18.0
KerasNLP version: 0.20.0


My task here is to take various tweets in text format and using NLP and deep learning, carry out binary classification to determine whether they are referencing a disaster or not. 

my data cleaning procedure is a simple dropna in the even any rows have missing data. 

I believe that an LSTM based architecture would be suitable because the long term memory aspect of LSTMs allow for long term dependence of some tokens to affect the interpretationa and final classification of the tweets. 

I could in the future try to implement dimensionality reductions schemes like Isomap. I am also curious if applying LDA and including the subsequent output could improve the model. 

In [42]:
import keras_tuner
import keras

In [43]:
df_train = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
df_test = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

print('Training Set Shape = {}'.format(df_train.shape))
print('Training Set Memory Usage = {:.2f} MB'.format(df_train.memory_usage().sum() / 1024**2))
print('Test Set Shape = {}'.format(df_test.shape))
print('Test Set Memory Usage = {:.2f} MB'.format(df_test.memory_usage().sum() / 1024**2))

Training Set Shape = (7613, 5)
Training Set Memory Usage = 0.29 MB
Test Set Shape = (3263, 4)
Test Set Memory Usage = 0.10 MB


In [44]:
#data cleaning 
df_train.dropna(axis=0,how='any')
print(df_train.shape)

(7613, 5)


In [45]:
BATCH_SIZE = 32
NUM_TRAINING_EXAMPLES = df_train.shape[0]
TRAIN_SPLIT = 0.8
VAL_SPLIT = 0.2
STEPS_PER_EPOCH = int(NUM_TRAINING_EXAMPLES)*TRAIN_SPLIT // BATCH_SIZE

EPOCHS = 2
AUTO = tf.data.experimental.AUTOTUNE

In [46]:
from sklearn.model_selection import train_test_split

X = df_train["text"]
y = df_train["target"]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

X_test = df_test["text"]

In [10]:
print(y.shape)

(7613,)


In [11]:
print(y_train.shape)

print(X_train.shape)

(6090,)
(6090,)


In [ ]:
print(X)

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(strip_accents=None,lowercase=False)

#vectorizer.get_feature_names_out()
print(X.shape)
xn = np.array(X_train)
#this single line after specifying the text data will do all the preprocessing. 
X_train_preprocessed = vectorizer.fit_transform(xn)#preprocessor(X_train)

X_val = vectorizer.fit_transform(X_val)
#x_val = preprocessor(X_val)

print(X_train_preprocessed)

(7613,)
<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 89916 stored elements and shape (6090, 22138)>
  Coords	Values
  (0, 10974)	0.2982535459993856
  (0, 13383)	0.054526920564421634
  (0, 15960)	0.12914874584715366
  (0, 20436)	0.2848486273381596
  (0, 19107)	0.2848486273381596
  (0, 17412)	0.20872378881345313
  (0, 5846)	0.24504456728569277
  (0, 5266)	0.2848486273381596
  (0, 212)	0.26193276752714595
  (0, 16174)	0.08432504935013056
  (0, 2217)	0.2390169077161323
  (0, 1692)	0.21897769600090522
  (0, 21257)	0.2122070703936803
  (0, 20838)	0.08484082575971075
  (0, 17742)	0.18990918366499138
  (0, 17979)	0.08515419869656861
  (0, 12060)	0.2982535459993856
  (0, 15898)	0.2848486273381596
  (0, 12066)	0.0947001431289348
  (0, 3056)	0.2982535459993856
  (1, 12774)	0.3356155786735253
  (1, 15044)	0.22147763598121362
  (1, 14602)	0.23365063602303468
  (1, 12526)	0.3015276599656971
  (1, 13095)	0.3205314343476497
  :	:
  (6088, 18854)	0.2612498294588477
  (6088, 19547)	0.20

In [13]:
e = X_train_preprocessed 
coo = e.tocoo()
indices = np.mat([coo.row, coo.col]).transpose()
e = indices
e = tf.SparseTensor(indices, coo.data, coo.shape)
e = tf.sparse.reorder(e)
e = tf.sparse.to_dense(e)
e = pd.DataFrame(e)

2025-05-01 16:18:26.139343: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [20]:
f = X_val 
coo = f.tocoo()
indices = np.mat([coo.row, coo.col]).transpose()
f = indices
f = tf.SparseTensor(indices, coo.data, coo.shape)
f = tf.sparse.reorder(f)
f = tf.sparse.to_dense(f)
f = pd.DataFrame(f)

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(strip_accents=None,lowercase=False)

#vectorizer.get_feature_names_out()

#this single line after specifying the text data will do all the preprocessing. 
X_test = vectorizer.fit_transform(X_test)#preprocessor(X_train)

X_test = X_test 

coo = X_test.tocoo()
indices = np.mat([coo.row, coo.col]).transpose()
X_test = indices
X_test = tf.SparseTensor(X_test, coo.data, coo.shape)
X_test = tf.sparse.reorder(X_test)
X_test  = tf.sparse.to_dense(X_test)
X_test = pd.DataFrame(X_test)

2025-05-02 17:57:57.589867: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [ ]:
print(e)

In [21]:
from sklearn.manifold import Isomap

#X = Isomap.fit_transform(e)


In [ ]:
lda = LatentDirichletAllocation(n_components=25,random_state=0)

In [ ]:
#this gives back topics. This

#lda = LatentDirichletAllocation(n_components=5,random_state=0)
#lda.fit(X_train_preprocessed['token_ids'])
#LatentDirichletAllocation(...)
# get topics for some given samples:
x_trainl = lda.transform(X_train_preprocessed['token_ids'])
X_vall = lda.transform(x_val['token_ids'])


In [ ]:
from sklearn.manifold import Isomap

#x_trainl = lda.transform(X_train_preprocessed['token_ids'])
#X_vall = lda.transform(x_val['token_ids'])


#xx = Isomap.fit_transform(X[:100])
#x = xx.transform(x)
#l= xx.transform(l)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.Input(shape=(25798,), dtype=tf.float64),#4360# 25
    tf.keras.layers.Reshape((25798, 1)),
    tf.keras.layers.LSTM(25, kernel_initializer='ones', use_bias=False, dtype=tf.float64),
    #tf.keras.layers.Dense(25,kernel_initializer='ones',use_bias=False),
    #tf.keras.layers.InputLayer(input_shape=(10),dtype=x.dtype),#put a 1 before the 9 later
    tf.keras.layers.Dense(50,kernel_initializer='ones', use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(75,kernel_initializer='ones', use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(100,kernel_initializer='ones', use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(125,kernel_initializer='ones', use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(150,kernel_initializer='ones',use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(175,kernel_initializer='ones',use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(200,kernel_initializer='ones',use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(225,kernel_initializer='ones',use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(250,kernel_initializer='ones',use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(275,kernel_initializer='ones',use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(300,kernel_initializer='ones',use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(275,kernel_initializer='ones',use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(250,kernel_initializer='ones',use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(225,kernel_initializer='ones',use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(200,kernel_initializer='ones',use_bias=False),
    #goal is to eventually replace the first dense layer with an LSTM layer
    #tf.keras.layers.LSTM
    #tf.keras.layers.TimeDistributed(Dense(vocabulary)))
    #tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(150,kernel_initializer='ones',use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(125,kernel_initializer='ones', use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(100,kernel_initializer='ones',use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(75,kernel_initializer='ones', use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(50,kernel_initializer='ones',use_bias=False),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(25,kernel_initializer='ones',use_bias=False,),
    tf.keras.layers.Dense(1,activation='sigmoid', kernel_regularizer='l1',kernel_initializer='ones',use_bias=False,)
])
model.compile(optimizer='Adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),metrics=['accuracy'])
model.fit(x=e,y=y,epochs=1750,validation_split=0.1)#batch_size=BATCH_SIZE,validation_data=(X_vall, y_val)

Prediction = model.predict(l)

In [47]:
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
import numpy as np

# Parameters
max_vocab_size = 10000
sequence_length = 100

# Create text vectorizer
vectorizer = TextVectorization(max_tokens=max_vocab_size, output_sequence_length=sequence_length)

# Fit on training text
vectorizer.adapt(X_train)  # text_data is a list/array of raw tweets

In [48]:
def build_model(hp=None):
    model = tf.keras.Sequential([
        tf.keras.Input(shape=(1,), dtype=tf.string),  # Input is raw text

        # Text vectorization layer
        vectorizer,

        # Embedding layer maps tokens to dense vectors
        tf.keras.layers.Embedding(input_dim=max_vocab_size, output_dim=128),

        # LSTM to process sequence
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=False)),

        tf.keras.layers.Dense(50,kernel_initializer='ones',activation=hp.Choice("activation", ["relu", "tanh"]), use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(75,kernel_initializer='ones', use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(100,kernel_initializer='ones', use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(125,kernel_initializer='ones', use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(150,kernel_initializer='ones',use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(175,kernel_initializer='ones',use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(200,kernel_initializer='ones',use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(225,kernel_initializer='ones',use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(250,kernel_initializer='ones',use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(275,kernel_initializer='ones',use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(300,kernel_initializer='ones',use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(275,kernel_initializer='ones',use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(250,kernel_initializer='ones',use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(225,kernel_initializer='ones',use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(200,kernel_initializer='ones',use_bias=False),
        #goal is to eventually replace the first dense layer with an LSTM layer
        #tf.keras.layers.LSTM
        #tf.keras.layers.TimeDistributed(Dense(vocabulary)))
        #tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(150,kernel_initializer='ones',use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(125,kernel_initializer='ones', use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(100,kernel_initializer='ones',use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(75,kernel_initializer='ones', use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(50,kernel_initializer='ones',use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(25,kernel_initializer='ones',use_bias=False,),
        tf.keras.layers.Dense(1,activation='sigmoid', kernel_regularizer='l1',kernel_initializer='ones',use_bias=False,)
    ])

    model.compile(
        optimizer='adam',
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=['accuracy']
    )
    return model

In [24]:
model = build_model()
#model.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val))



Epoch 1/5
191/191 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - accuracy: 0.6767 - loss: 210.1533

ValueError: Exception encountered when calling TextVectorization.call().

[1mWhen using `TextVectorization` to tokenize strings, the input rank must be 1 or the last shape dimension must be 1. Received: inputs.shape=(None, 8338) with rank=2[0m

Arguments received by TextVectorization.call():
  • inputs=tf.Tensor(shape=(None, 8338), dtype=string)

In [49]:
tuner = keras_tuner.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=5,
    directory='my_dir',
    project_name='my_project',
    overwrite=True
)
tuner.search(X_train, y_train, epochs=5, validation_data=(X_val, y_val))
best_hp = tuner.get_best_hyperparameters(1)[0]
print(best_hp)

Trial 2 Complete [00h 02m 01s]
val_loss: 7.566357612609863

Best val_loss So Far: 2.833733081817627
Total elapsed time: 00h 04m 03s


In [50]:
best_model = build_model(best_hp)

# Optional: retrain the best model using all data or just validation set
best_model.fit(X_train, y_train, epochs=1, validation_data=(X_val, y_val))

191/191 ━━━━━━━━━━━━━━━━━━━━ 47s 101ms/step - accuracy: 0.6534 - loss: 172.6660 - val_accuracy: 0.5883 - val_loss: 942.2491


In [37]:
sample_submission = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")
sample_submission.head()

sample_submission["target"] = np.argmax(best_model.predict(X_test), axis=1)

102/102 ━━━━━━━━━━━━━━━━━━━━ 5s 35ms/step


In [39]:
sample_submission.to_csv("submission.csv", index=False)

In [27]:
def build_model(hp): 
    model = tf.keras.Sequential([
        tf.keras.Input(shape=(1,25798), dtype=tf.float64),#4360# 25
        #tf.keras.layers.Reshape((25798)),
        tf.keras.layers.LSTM(25, kernel_initializer='ones', use_bias=False, dtype=tf.float64),
        #tf.keras.layers.Dense(25,kernel_initializer='ones',use_bias=False),
        #tf.keras.layers.InputLayer(input_shape=(10),dtype=x.dtype),#put a 1 before the 9 later
        tf.keras.layers.Dense(50,kernel_initializer='ones', use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(75,kernel_initializer='ones', use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(100,kernel_initializer='ones', use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(125,kernel_initializer='ones', use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(150,kernel_initializer='ones',use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(175,kernel_initializer='ones',use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(200,kernel_initializer='ones',use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(225,kernel_initializer='ones',use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(250,kernel_initializer='ones',use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(275,kernel_initializer='ones',use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(300,kernel_initializer='ones',use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(275,kernel_initializer='ones',use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(250,kernel_initializer='ones',use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(225,kernel_initializer='ones',use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(200,kernel_initializer='ones',use_bias=False),
        #goal is to eventually replace the first dense layer with an LSTM layer
        #tf.keras.layers.LSTM
        #tf.keras.layers.TimeDistributed(Dense(vocabulary)))
        #tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(150,kernel_initializer='ones',use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(125,kernel_initializer='ones', use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(100,kernel_initializer='ones',use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(75,kernel_initializer='ones', use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(50,kernel_initializer='ones',use_bias=False),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(25,kernel_initializer='ones',use_bias=False,),
        tf.keras.layers.Dense(1,activation='sigmoid', kernel_regularizer='l1',kernel_initializer='ones',use_bias=False,)
    ])
    model.compile(optimizer='Adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),metrics=['accuracy'])
    return model

In [12]:
build_model(keras_tuner.HyperParameters())

<Sequential name=sequential, built=True>

In [13]:
tuner = keras_tuner.RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=5,
    directory='my_dir',
    project_name='my_project',
    overwrite=True
)
tuner.search(e, y_train, epochs=5, validation_data=(f, y_val))
best_hp = tuner.get_best_hyperparameters(1)[0]
best_model = build_model(best_hp)

# Optional: retrain the best model using all data or just validation set
best_model.fit(e, y_train, epochs=5, validation_data=(f, y_val))

NameError: name 'e' is not defined

In [47]:
tunerh = keras_tuner.Hyperband(
    hypermodel=build_model,
    objective="val_accuracy",
    max_epochs=2,
    factor=3,
    hyperband_iterations=1,
    #distribution_strategy=tf.distribute.MirroredStrategy(),
    #directory="results_dir",
    #project_name="mnist",
    #overwrite=True,
)

Reloading Tuner from ./untitled_project/tuner0.json


RuntimeError: Error reloading `Oracle` from existing project. If you did not mean to reload from an existing project, change the `project_name` or pass `overwrite=True` when creating the `Tuner`. Found existing project at: ./untitled_project

In [48]:
tuner.search(
    e,#x_train,
    y_train,
    steps_per_epoch=600,
    validation_data=(f, y_val),
    validation_steps=100,
    callbacks=[keras.callbacks.EarlyStopping("val_accuracy")],
)

In [49]:
tuner.search(e, y_train, epochs=2, validation_data=(f, y_val))

In [31]:
#NLP_1_model = tuner.get_best_models(num_models=1)#[0]
#NLP_1_model.summary()
#print(NLP_1_model)

[]


In [50]:
models = tuner.get_best_models(num_models=2)
best_model = models[0]
best_model.summary()

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = './untitled_project/trial_0/checkpoint.weights.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
import numpy as np
from sklearn import random_projection
X = np.random.rand(100, 10000)
print(X.shape)# random projection can reduce the shape of something from 100,10000 to 100,3947. 
#memory requirements can be reduced further if it is sparse projection

#transformer = random_projection.SparseRandomProjection()
#X_new = transformer.fit_transform(X)

note: Do NOT use Transformer based models. USe a custom LSTM neural network based architecture to satisfy the rubric

Use a tfidf vectorizer instead. Look into possibly using word2vec as well.  

Should also determine whether isomap, or TSNE would be effective. 

List of confirmed core preprocessing: 

standard scalar, vectorizer

Also required by assignment: hyperparameter tuning different architectures 

List of preprocessing things to consider: 

, LDA, random projection, isomap or TSNE

also implement gaussian random projection

GOals accomplished

neural network with LSTM is working

vectorizer is working

